In [144]:
import pyodbc
import pandas as pd
import datetime as dt
import numpy as np
from datetime import timedelta
from datetime import datetime
from dateutil import parser as dtparser

! pip install pandasql
import pandasql as ps

def dt_form(x):
    return dt.date.isoformat(x)


#Code to connect to data warehouse. driver = type of server. 
# server = sql server's name and database = which database on the sql server you want to access.
# trusted connection = yes just means you won't have a security error thrown.
bass = pyodbc.connect(driver = '{SQL SERVER}', 
                            Server = 'shcd-chi-dw1',
                            Database = 'DW_Bass',
                            trusted_connection = 'yes')



In [155]:
# SQL query to create main table for Bass
bassquery = """  \
select fm.customerkey, count(distinct ft.transactionkey) as trans, sum(netsalesamount) as sales
                  , count(distinct case when ic_firsttransactionflag = 0 then transactionkey end) as subtrans
				  , sum(case when ic_firsttransactionflag = 0 then netsalesamount end) as subsales
				  , count(distinct fr.rewardnumber) as rewards
				  , count(distinct case when rewardamountused > 0 then fr.rewardnumber end) as redemps
from fact_member fm
left join fact_transaction ft on fm.customerkey = ft.customerkey
inner join fact_Membership fms on fm.customerkey = fms.customerkey
left join fact_loyaltyreward fr on fms.CustomerMembershipKey = fr.CustomerMembershipKey
where convert(date,enrolldate) >= '2/3/2019'
group by fm.customerkey
order by fm.customerkey """

# Creation of Bass Table
bass_table = pd.read_sql_query(bassquery,bass)

bass_table.head(10)

,customerkey,trans,sales,subtrans,subsales,rewards,redemps
0,4383744,2,184.82,1,-7.00,1,0
1,4385153,0,NaN,0,NaN,0,0
2,4385155,2,185.42,1,108.71,1,0
3,4385156,3,150.12,2,52.62,1,0
4,4385157,1,24.00,0,NaN,0,0
5,4385158,3,361.27,2,5.18,1,0
6,4385160,1,23.90,0,NaN,1,0
7,4385161,1,51.04,0,NaN,0,0
8,4385162,1,25.38,0,NaN,0,0
9,4385163,1,95.98,0,NaN,0,0


In [73]:
# SQL statement to count customers in bass_table and run sql on bass_table dataframe

sub1 = """  \
select count(distinct customerkey) as customers
from bass_table
"""

total_cust= ps.sqldf(sub1,locals())
total_cust

,customers
0,231808


In [76]:
# SQL statement to count multi-purchase customers in bass_table and run sql on bass_table dataframe

sub2 = """ \
select count(distinct customerkey) as customers, sum(subtrans) as subtrans, sum(subsales) as subsales
from bass_table
where subtrans >= 1
"""

multi=ps.sqldf(sub2,locals())

In [156]:
# Create separate dataset for multi-purchase custs 

bass_multi = bass_table[bass_table['subtrans'] >=1]

bass_multi.head(10)

,customerkey,trans,sales,subtrans,subsales,rewards,redemps
0,4383744,2,184.82,1,-7.00,1,0
2,4385155,2,185.42,1,108.71,1,0
3,4385156,3,150.12,2,52.62,1,0
5,4385158,3,361.27,2,5.18,1,0
24,4385178,3,149.90,2,79.98,1,0
26,4385180,4,69.69,3,19.14,1,0
27,4385181,3,151.28,2,90.57,1,0
28,4385182,2,23.99,1,0.00,0,0
45,4385199,2,65.37,1,52.49,0,0
66,4385220,3,112.50,2,58.61,1,0


In [60]:
# Set a flag for multi-purchase custs

bass_table['Multi_Fl'] = [1 if x >= 1 else 0 for x in bass_table['subtrans']]

bass_table.head(10)

,customerkey,trans,sales,subtrans,subsales,rewards,redemps,Multi_Fl
0,4383744,2,184.82,1,-7.00,1,0,1
1,4385153,0,NaN,0,NaN,0,0,0
2,4385155,2,185.42,1,108.71,1,0,1
3,4385156,3,150.12,2,52.62,1,0,1
4,4385157,1,24.00,0,NaN,0,0,0
5,4385158,3,361.27,2,5.18,1,0,1
6,4385160,1,23.90,0,NaN,1,0,0
7,4385161,1,51.04,0,NaN,0,0,0
8,4385162,1,25.38,0,NaN,0,0,0
9,4385163,1,95.98,0,NaN,0,0,0


In [61]:
# Replace NaN with 0

bass_table.fillna({'sales':0, 'subsales':0}, inplace = True)


bass_table.head(10)

,customerkey,trans,sales,subtrans,subsales,rewards,redemps,Multi_Fl
0,4383744,2,184.82,1,-7.00,1,0,1
1,4385153,0,0.00,0,0.00,0,0,0
2,4385155,2,185.42,1,108.71,1,0,1
3,4385156,3,150.12,2,52.62,1,0,1
4,4385157,1,24.00,0,0.00,0,0,0
5,4385158,3,361.27,2,5.18,1,0,1
6,4385160,1,23.90,0,0.00,1,0,0
7,4385161,1,51.04,0,0.00,0,0,0
8,4385162,1,25.38,0,0.00,0,0,0
9,4385163,1,95.98,0,0.00,0,0,0


In [78]:
# SQL statement enrolls by month

enr = """ \
select FY_No as year, FP_No as month, count(distinct customerkey) as customers
from fact_member fm
inner join dim_Store ds on fm.EnrollStoreKey = ds.StoreKey
inner join dim_Date dd on fm.EnrollDate = dd.SQLDateStamp
where dd.FY_No in (2018, 2019)
and ds.Active = 1
and ds.OpenDate <= '2/4/2018'
group by FY_No, fp_no
order by FY_No, fp_no

"""

enr_mo = pd.read_sql_query(enr,connection)

enr_mo

,year,month,customers
0,2018,1,27919
1,2018,2,37050
2,2018,3,31172
3,2018,4,35806
4,2018,5,49227
5,2018,6,38255
6,2018,7,36702
7,2018,8,43551
8,2018,9,35833
9,2018,10,54998


In [79]:
# Export dataframes to excel/csv

enr_mo.to_csv('Q:\\Confidential\\AMR\\072019 - DM Conference\\enrbymonth_bass.csv')

enr_mo.to_excel('Q:\\Confidential\\AMR\\072019 - DM Conference\\enrbymonth_bass.xlsx')
total_cust.to_excel('Q:\\Confidential\\AMR\\072019 - DM Conference\\totalcust_bass.xlsx')



# Export dataframes to excel/csv
multi.to_csv('Q:\\Confidential\\AMR\\072019 - DM Conference\\multicust_bass.csv')
multi.to_excel('Q:\\Confidential\\AMR\\072019 - DM Conference\\multicust_bass.xlsx')


In [74]:
total_cust

,customers
0,231808


In [80]:
# SQL statement enrolled 18 purchased in 19

sqlenr2018pur19 = """ \
select FY_No as year, FP_No as month, count(distinct fm.customerkey) as customers, COUNT(distinct case when trans19>=1 then aa.customerkey end) as multi
from fact_member fm
inner join dim_Date dd on fm.EnrollDate = dd.SQLDateStamp
left join 
   (select customerkey, COUNT(distinct transactionkey) as trans19 
   from fact_Transaction ft 
   inner join dim_date dd on ft.transactiondate = dd.sqldatestamp 
   where dd.FY_No in (2019)
   group by customerkey) aa on fm.CustomerKey = aa.CustomerKey
where dd.FY_No in (2018)
group by FY_No, fp_no
order by FY_No, fp_no

"""

enr2018pur19 = pd.read_sql_query(sqlenr2018pur19,connection)

enr2018pur19

,year,month,customers,multi
0,2018,1,32850,3089
1,2018,2,43125,4134
2,2018,3,35898,3525
3,2018,4,41110,4083
4,2018,5,55826,5553
5,2018,6,43334,4154
6,2018,7,41527,3875
7,2018,8,49410,4778
8,2018,9,39847,4160
9,2018,10,58589,5484


In [82]:
# SQL statement enrolled 18 purchased in 19 - comp only

sqlenr2018pur19comp = """ \
select FY_No as year, FP_No as month, count(distinct fm.customerkey) as customers, COUNT(distinct case when trans19>=1 then aa.customerkey end) as multi
from fact_member fm
inner join dim_Date dd on fm.EnrollDate = dd.SQLDateStamp
inner join dim_Store ds on fm.EnrollStoreKey = ds.StoreKey
left join 
   (select customerkey, COUNT(distinct transactionkey) as trans19 
   from fact_Transaction ft 
   inner join dim_date dd on ft.transactiondate = dd.sqldatestamp 
   where dd.FY_No in (2019)
   group by customerkey) aa on fm.CustomerKey = aa.CustomerKey
where dd.FY_No in (2018)
and ds.Active = 1
and ds.OpenDate <= '2/4/2018'
group by FY_No, fp_no
order by FY_No, fp_no


"""

enr2018pur19comp = pd.read_sql_query(sqlenr2018pur19comp,connection)

enr2018pur19comp

,year,month,customers,multi
0,2018,1,27919,2962
1,2018,2,37050,3966
2,2018,3,31172,3398
3,2018,4,35806,3931
4,2018,5,49227,5372
5,2018,6,38255,4016
6,2018,7,36702,3731
7,2018,8,43551,4611
8,2018,9,35833,4031
9,2018,10,54998,5362


In [84]:
# SQL statement enrolled 18 purchased in 19 - comp only -- non SQL

sqlenr2018 = """ \
select distinct fm.customerkey
from fact_member fm
inner join dim_Date dd on fm.EnrollDate = dd.SQLDateStamp
inner join dim_Store ds on fm.EnrollStoreKey = ds.StoreKey
where dd.FY_No in (2018)
and ds.Active = 1
and ds.OpenDate <= '2/4/2018'

"""

enr2018 = pd.read_sql_query(sqlenr2018,connection)
enr2018.head(10)

,customerkey
0,4127353
1,4144333
2,4178293
3,4184127
4,4110373
5,4218087
6,4252047
7,4201107
8,4235067
9,4269027


In [85]:
# SQL statement enrolled 18 purchased in 19 - comp only -- non SQL

sqlpur2019 = """ \
select customerkey, COUNT(distinct transactionkey) as trans19 
   from fact_Transaction ft 
   inner join dim_date dd on ft.transactiondate = dd.sqldatestamp 
   where dd.FY_No in (2019)
   group by customerkey

"""

pur2019 = pd.read_sql_query(sqlpur2019,connection)
pur2019.head(10)

,customerkey,trans19
0,2678764.0,3
1,3856978.0,2
2,3396146.0,1
3,3425861.0,1
4,4462401.0,1
5,459765.0,1
6,4587095.0,2
7,3082822.0,1
8,4591340.0,1
9,4604075.0,1


In [148]:
# Merge Statement - Left Join  (left_on and right_on are optional in this case, validate also optional)

ex_merge = pd.merge(enr2018, pur2019, how = 'left', left_on = 'customerkey', right_on = 'customerkey')
ex_merge = pd.merge(enr2018, pur2019, how = 'left', on = 'customerkey', validate="one_to_many")


# Merge Statement - Inner Join

ex_merge = pd.merge(enr2018, pur2019, how = 'inner', on = 'customerkey')
ex_merge = pd.merge(enr2018, pur2019, how = 'inner', validate="one_to_one")

ex_merge.head(10)

,customerkey,trans19
0,4218087,1
1,4269027,1
2,4168283,3
3,4243680,1
4,4165750,1
5,4360268,2
6,4377248,1
7,4051956,2
8,4037509,1
9,3910943,3


In [101]:
sqlpur2019m = """ \
select COUNT(distinct customerkey) as act19 
   from ex_merge

"""

pur19=ps.sqldf(sqlpur2019m,locals())
pur19.head(10)

,act19
0,52846


In [107]:
# Save dataframe via pickle, and pull back in via pickle


bass_table.to_pickle('Q:\\Confidential\\AMR\\072019 - DM Conference\\bass_table.pkl')
imported = pd.read_pickle('Q:\\Confidential\\AMR\\072019 - DM Conference\\bass_table.pkl')


In [111]:
bass_table.head(10)

,customerkey,trans,sales,subtrans,subsales,rewards,redemps
0,4383744,2,184.82,1,-7.00,1,0
1,4385153,0,NaN,0,NaN,0,0
2,4385155,2,185.42,1,108.71,1,0
3,4385156,3,150.12,2,52.62,1,0
4,4385157,1,24.00,0,NaN,0,0
5,4385158,3,361.27,2,5.18,1,0
6,4385160,1,23.90,0,NaN,1,0
7,4385161,1,51.04,0,NaN,0,0
8,4385162,1,25.38,0,NaN,0,0
9,4385163,1,95.98,0,NaN,0,0


In [112]:
imported.head(10)

,customerkey,trans,sales,subtrans,subsales,rewards,redemps
0,4383744,2,184.82,1,-7.00,1,0
1,4385153,0,NaN,0,NaN,0,0
2,4385155,2,185.42,1,108.71,1,0
3,4385156,3,150.12,2,52.62,1,0
4,4385157,1,24.00,0,NaN,0,0
5,4385158,3,361.27,2,5.18,1,0
6,4385160,1,23.90,0,NaN,1,0
7,4385161,1,51.04,0,NaN,0,0
8,4385162,1,25.38,0,NaN,0,0
9,4385163,1,95.98,0,NaN,0,0


In [116]:
bass_multi.head(10)

,customerkey,trans,sales,subtrans,subsales,rewards,redemps
0,4383744,2,184.82,1,-7.00,1,0
2,4385155,2,185.42,1,108.71,1,0
3,4385156,3,150.12,2,52.62,1,0
5,4385158,3,361.27,2,5.18,1,0
24,4385178,3,149.90,2,79.98,1,0
26,4385180,4,69.69,3,19.14,1,0
27,4385181,3,151.28,2,90.57,1,0
28,4385182,2,23.99,1,0.00,0,0
45,4385199,2,65.37,1,52.49,0,0
66,4385220,3,112.50,2,58.61,1,0


In [150]:
# Both of these will get you mean of trans

bass_multi["trans"].mean()
bass_multi.trans.mean()

2.3909648182241194

In [151]:
# Mean of sales by trans

bass_multi.groupby('trans')['sales'].mean()

# Mean of sales and subsales by trans
bass_multi.groupby('trans')['sales','subsales'].mean()

,sales,subsales
trans,,
1,83.608743,83.608743
2,98.004386,25.124671
3,161.646622,75.415690
4,229.866439,126.913517
5,298.813439,192.533703
6,431.195069,306.887051
7,516.302660,368.171379
8,536.958554,375.004819
9,617.147222,487.245370


In [132]:
print(bass_table[bass_table['trans'] ==58])

       customerkey  trans    sales  subtrans  subsales  rewards  redemps
70270      4455982     58  2768.93        57   2709.47        0        0


In [152]:
# Freq (customers) of trans 
bass_multi.groupby('trans')['customerkey'].count()

trans
1      2268
2     34272
3      8344
4      2718
5       948
6       434
7       203
8        83
9        54
10       22
11       21
12       11
13       17
14        6
15        4
16        3
17        2
18        4
19        3
20        1
21        2
22        2
24        1
28        2
30        1
32        1
58        1
87        1
Name: customerkey, dtype: int64

In [167]:
ranges = [0,1,2,3,6,9, np.inf]
bass_multi['customerkey'].groupby(pd.cut(bass_multi.trans, ranges)).count()

trans
(0.0, 1.0]     2302
(1.0, 2.0]    34729
(2.0, 3.0]     8489
(3.0, 6.0]     4167
(6.0, 9.0]      353
(9.0, inf]      107
Name: customerkey, dtype: int64

In [170]:
# FREQUENCY BY BINS: OPTION 1 

ranges = [0,1,2,3,6,9, np.inf]
bass_multi.customerkey.groupby(pd.cut(bass_multi.trans, ranges)).count()

trans
(0.0, 1.0]     2302
(1.0, 2.0]    34729
(2.0, 3.0]     8489
(3.0, 6.0]     4167
(6.0, 9.0]      353
(9.0, inf]      107
Name: customerkey, dtype: int64

In [171]:
# FREQUENCY BY BINS: OPTION 2 

bins = [0,1,2,3,6,9, np.inf]
labels = ['1','2','3','4-6','7-9','10+']
bass_multi.customerkey.groupby(pd.cut(bass_multi.trans, bins=bins, labels=labels)).count()

trans
1       2302
2      34729
3       8489
4-6     4167
7-9      353
10+      107
Name: customerkey, dtype: int64

In [142]:
print(bass_table[bass_table['trans'] ==13])

        customerkey  trans    sales  subtrans  subsales  rewards  redemps
4910        4390552     13  3720.36        12   3494.56        2        1
22927       4408608     13  1982.91        12   1839.51        3        3
25069       4410751     13   900.66        12    725.52        3        0
28182       4413865     13  2433.03        12   2152.59        3        0
36447       4422134     13  2308.88        12   1928.88        4        3
41801       4427489     13  1608.75        12   1599.45        3        0
49414       4435108     13  1516.32        12   1222.65        3        2
60143       4445839     13   672.20        12    346.36        2        0
66240       4451947     13   100.85        12     70.86        1        1
85657       4471390     13   566.08        12    482.50        2        0
89631       4475365     13  2337.51        12   1962.63        3        3
104811      4490547     13  5579.00        12   3853.16        2        1
120775      4506511     13  1415.60   

In [149]:
ex_merge = pd.merge(enr2018, pur2019, how = 'left', on = 'customerkey', validate="one_to_many")

In [172]:
ex_merge.head(10)

,customerkey,trans19
0,4127353,NaN
1,4144333,NaN
2,4178293,NaN
3,4184127,NaN
4,4110373,NaN
5,4218087,1.0
6,4252047,NaN
7,4201107,NaN
8,4235067,NaN
9,4269027,1.0


In [178]:
ex_merge.sort_values(by=['trans19','customerkey'], ascending=[False,True])

,customerkey,trans19
458705,4356351,45.0
362859,4363313,44.0
17808,3853794,35.0
60096,4068831,34.0
18479,3926655,30.0
37751,4228014,29.0
397739,4027354,28.0
170105,4063568,28.0
348115,4093910,28.0
174905,3922045,27.0


In [182]:
# SQL query to create main table for Bass
bassquery = """  \
select distinct customerkey, transactiondate
from fact_transaction
where customerkey is not null
order by customerkey, transactiondate """

# Creation of Days Table
days = pd.read_sql_query(bassquery,bass)

# Difference in transactiondate from prior row
days['dbv'] = days.transactiondate.diff()

# Removes First Row for each Customerkey
days2 = days[days['customerkey'].duplicated()]


In [201]:
# FREQUENCY BY BINS: OPTION 2 
bins = [0,4,9,19,29,39,49,59,89,119,149,179, np.inf]
labels = ['1-4','5-9','10-19','20-29','30-39','40-49','50-59','60-89','90-119','120-149','150-179','180+']
days2.customerkey.groupby(pd.cut(days2.dbv.dt.days, bins=bins, labels=labels)).count()

dbv
1-4         549235
5-9         400708
10-19       525369
20-29       441728
30-39       315129
40-49       279317
50-59       225884
60-89       548323
90-119      429655
120-149     301168
150-179     241186
180+       1825289
Name: customerkey, dtype: int64

In [206]:
# Mean of days between

days2.dbv.dt.days.mean()

178.49739889472136

In [207]:
# Median of days between

days2.dbv.dt.days.median()

75.0

In [217]:
# count, mean, std, min, 25, 50, 75, max of days between

days2.dbv.describe()

count                     6082991
mean     178 days 11:56:15.264503
std      270 days 16:36:29.938323
min               1 days 00:00:00
25%              20 days 00:00:00
50%              75 days 00:00:00
75%             225 days 00:00:00
max            3467 days 00:00:00
Name: dbv, dtype: object